In [3]:

import os
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
import time
from dotenv import load_dotenv
import bs4
from langchain_groq import ChatGroq

load_dotenv()

## load the Groq API key
groq_api_key = os.environ['GROQ_API_KEY']

In [4]:

embeddings = OllamaEmbeddings()
loader=WebBaseLoader("https://react.dev/learn")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=250)
final_documents = text_splitter.split_documents(docs)
vectors = FAISS.from_documents(final_documents, embeddings)

In [5]:

llm=ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")



prompt=ChatPromptTemplate.from_template(
"""
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question
<context>
{context}
<context>
Questions:{input}

"""
)
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectors.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)



In [6]:

response=retrieval_chain.invoke({"input":'create 5 random objective type questions which have( 4 options out of 1 is correct) from the context given.The questions should be on methods and functions in react.  give its answer also. the questions should be based on react programming not on its general awareness. questions should not be on its features, pre requisites, compilers, comparisons with other languages, authors. The questions should be a little above beginner level '})
print(response)

KeyboardInterrupt: 